# API - Properties

This notebook illustrates the main features of pyNeVer for defining a specification to create a verification benchmark

## The _properties_ module

The module _properties_ contains the classes to create and load verification properties

In [4]:
import torch
from pynever.strategies.verification import properties

# Let us reconsider the 2D neural network from the previous example
from pynever import networks
from pynever import nodes

w = torch.Tensor([[1, 1], [-1, 1]])
b = torch.zeros(2)

nn = networks.SequentialNetwork('nn', 'x')
nn.append_node(nodes.FullyConnectedNode('fc', (2,), 2, weight=w, bias=b))
nn.append_node(nodes.ReLUNode('relu', (2,)))

The base class __NeverProperty__ defines the constraints on the input and on the output as a system of linear inequalities for the input and a set of systems for the output.

The components are the following:
- _in_coef_mat_ is the coefficient matrix for the input variables
- _in_bias_mat_ is the bias array for the input variables
- _out_coef_mat_ is a list of coefficient matrices for the output variables
- _out_bias_mat_ is a list of bias arrays for the output variables

The difference between inputs and outputs is that - as output specifications define the unsafe zone - the constraints on the output can be expressed as a disjunction, each a component of the list.

In [ ]:
# Input specification: -1 <= x0 <= 1, -1 <= x1 <= 1
# expressed in normal form as Cx <= d
in_coef_mat = torch.Tensor([[1, 0], [-1, 0], [0, 1], [0, -1]])
in_bias_mat = torch.Tensor([1, 1, 1, 1])

# Output specification: y0 >= 2 (single statement)
out_coef_mat = [torch.Tensor([[-1, 0]])]
out_bias_mat = [torch.Tensor([-2])]

# Create the property
my_prop = properties.NeverProperty(in_coef_mat, in_bias_mat, out_coef_mat, out_bias_mat)

It is also possible to directly load a VNNLIB specification from file using the class __VnnLibProperty__. The file extension can be .txt, .vnnlib, .smt2 or any plaintext format

In [ ]:
my_vnnlib_prop = properties.VnnLibProperty('prop.vnnlib')

# VnnLibProperty subclasses NeverProperty: the matrices are read from a file
my_vnnlib_prop.in_coef_mat == in_coef_mat

To create local robustness properties, one can load a sample and create the L-inf noise ball for each variable